# Cleaning Dataset

The goal of this notebook is to clean two datasets — review and meta — to prepare them for further analysis.
We removed fields that are not useful for exploratory data analysis (EDA) or subsequent modeling, and standardized data types and formats to make analysis easier.

### Dataset Files
* Review dataset
* Meta dataset (product information)

### Remarks
* Each dataset is divided by main category. In this notebook, we demonstrate our cleaning logic using the “All_Beauty” category.
* We observed that schema structures and list formats differ across categories. Therefore, we enforced consistent field names and ordering at the end of the cleaning process to ensure standardization.

In [1]:
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")

# You can redirect the bucket directory to where you stored your datasets!
data = "gs://" + bucket + "/Datasets/"
print(data)

gs://qst843-spark_phs/Datasets/


In [12]:
# Load Review Data
df = spark.read.json(data + "All_Beauty.jsonl")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



In [13]:
# Review dataset cleaning
from pyspark.sql.functions import col, when, length, regexp_replace, trim, to_timestamp, size, coalesce, lit, date_format, lower

## Drop asin
df = df.drop("asin")

## New Column "image_exist" to flag whether review contains image or not
df = df.withColumn(
    "review_image",
    (coalesce(size(col("images")), lit(0)) > 0)
)

## Drop image
df = df.drop("images")

## Type re-organize
df = (df
      .withColumn("helpful_vote", col("helpful_vote").cast("int"))
      .withColumn("rating", col("rating").cast("int"))
      .withColumn("verified_purchase", col("verified_purchase").cast("boolean"))
)

## timestamp(ms) → timestamp(sec) -> yyyy-MM-dd HH:mm:ss format
df = df.withColumn("timestamp", to_timestamp((col("timestamp")/1000).cast("double")))
df = df.withColumn("timestamp", date_format(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))

## String text clean(trim)
df = (df
      .withColumn("review_title", trim(regexp_replace(col("title"), r"\s+", " ")))
      .withColumn("review_text",  trim(regexp_replace(col("text"),  r"\s+", " ")))
)
df = df.drop("title", "text")

## Rating check(1-5)
df = df.where((col("rating") >= 1.0) & (col("rating") <= 5.0))

## Drop duplicates
df = df.dropDuplicates(["user_id", "parent_asin", "timestamp", "review_title"])



## Column order reconstruct
final_cols = [
    "user_id",
    "parent_asin",
    "timestamp",
    "rating",
    "review_title",
    "review_text",
    "helpful_vote",
    "verified_purchase",
    "review_image",
]
df = df.select(*final_cols)


In [14]:
df.show(20)

+--------------------+-----------+-------------------+------+--------------------+--------------------+------------+-----------------+------------+
|             user_id|parent_asin|          timestamp|rating|        review_title|         review_text|helpful_vote|verified_purchase|review_image|
+--------------------+-----------+-------------------+------+--------------------+--------------------+------------+-----------------+------------+
|AE223RTPWMG62AWDZ...| B07J5KZY8N|2019-10-14 15:55:19|     4|More than I expected|I didn't have muc...|           0|             true|       false|
|AE223YNB742YCE2HS...| B00AX3YNPC|2014-07-21 09:18:37|     5|          Five Stars|Works well and fi...|           0|             true|       false|
|AE224HM2QAW5TTSDL...| B0001HYKBC|2004-10-12 14:19:20|     4|Really As Close a...|I found the Nivea...|          22|            false|       false|
|AE225IKAL7QABBT2B...| B06XZJTCYW|2019-12-24 13:53:30|     1|                 Meh|I haven’t really ...|         

In [15]:
# Load Meta Data
spark.conf.set("spark.sql.caseSensitive", "true")
df_meta = spark.read.json(data + "meta_All_Beauty.jsonl")

In [16]:
# Meta dataset cleaning
from pyspark.sql.functions import transform, array_join

## New Column "image_exist" to flag whether review contains image or not
df_meta = df_meta.withColumn(
    "product_video",
    (coalesce(size(col("videos")), lit(0)) > 0)
)

## New Column "image_exist" to flag whether review contains image or not
df_meta = df_meta.withColumn(
    "product_image",
    (coalesce(size(col("images")), lit(0)) > 0)
)

## Drop images / videos
df_meta = df_meta.drop("images", "videos", "details")

## String text clean(trim) - clean multiple spaces & lowercase (store)
df_meta = (df_meta
    .withColumn("features",
        transform(
            col("features"),
            lambda x: (trim(regexp_replace(x, r"\s+", " ")))
        )
    )
    .withColumn("description",
        transform(
            col("description"),
            lambda x: trim(regexp_replace(x, r"\s+", " "))
        )
    )
)

df_meta = (df_meta
           .withColumn("product_name",  trim(regexp_replace(col("title"),  r"\s+", " ")))
           .withColumn("store", lower(trim(regexp_replace(col("store"),  r"\s+", " "))))
)

df_meta = df_meta.drop("title")

## Column order reconstruct
meta_final_cols = [
    "parent_asin",
    "product_name",
    "main_category",
    "categories",
    "price",
    "features",
    "description",
    "average_rating",
    "rating_number",
    "store",
    "product_image",
    "product_video"
]

df_meta = df_meta.select(*meta_final_cols)

In [17]:
df_meta.printSchema()
df_meta.show(1)

root
 |-- parent_asin: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- price: double (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- average_rating: double (nullable = true)
 |-- rating_number: long (nullable = true)
 |-- store: string (nullable = true)
 |-- product_image: boolean (nullable = false)
 |-- product_video: boolean (nullable = false)

+-----------+--------------------+-------------+----------+-----+--------+-----------+--------------+-------------+---------------+-------------+-------------+
|parent_asin|        product_name|main_category|categories|price|features|description|average_rating|rating_number|          store|product_image|product_video|
+-----------+------

In [18]:
# Left join two datasets (Review & Meta)
from pyspark.sql.functions import col

df_joined = df.join(
    df_meta,                    # the metadata table
    on="parent_asin",           # join key
    how="left"                  # keep all rows from df
)

df_joined.show(3)

+-----------+--------------------+-------------------+------+--------------------+--------------------+------------+-----------------+------------+--------------------+-------------+----------+------+--------------------+--------------------+--------------+-------------+-------+-------------+-------------+
|parent_asin|             user_id|          timestamp|rating|        review_title|         review_text|helpful_vote|verified_purchase|review_image|        product_name|main_category|categories| price|            features|         description|average_rating|rating_number|  store|product_image|product_video|
+-----------+--------------------+-------------------+------+--------------------+--------------------+------------+-----------------+------------+--------------------+-------------+----------+------+--------------------+--------------------+--------------+-------------+-------+-------------+-------------+
| B07J5KZY8N|AE223RTPWMG62AWDZ...|2019-10-14 15:55:19|     4|More than I exp